In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import matplotlib.pyplot as plt
import numpy as np

# Remove any magic commands like %matplotlib inline

# Define the introduction content as a raw string
introduction = r"""
# Gardiner Expressway Rehabilitation Simulator

## Game Introduction

You are the project manager for the rehabilitation of the Gardiner Expressway between Dufferin Street and Strachan Avenue. The project involves the following tasks:

---

### **Project Tasks Overview**

The project consists of the following tasks in sequence (critical path):

1. **Task 1: Site Preparation**
   - **Equipment Required**: None
   - **Total Work Units**: 100 units
   - **Additional Labor Required**: 5 workers
   - **Note**: Additional workers are required to start the task but do not contribute to productivity.
   - **Each productive worker can complete 1 unit per hour.**

2. **Task 2: Placing Beams**
   - **Equipment Required**: Crane (any type)
   - **Total Work Units**: 300 units
   - **Additional Labor Required**: 5 workers
   - **Note**: Additional workers are required to start the task but do not contribute to productivity.

3. **Task 3: Placing Concrete Deck**
   - **Equipment Required**: Concrete Mixer (any type)
   - **Total Work Units**: 2300 units
   - **Additional Labor Required**: 8 workers
   - **Note**: Additional workers are required to start the task but do not contribute to productivity.

4. **Task 4: Applying New Traffic Management System**
   - **Equipment Required**: None
   - **Total Work Units**: 180 units
   - **Maintenance Workers**: 5 workers (do not contribute to productivity)
   - **Each productive worker can complete 1 unit per hour.**

5. **Task 5: Installing Railing and Lighting**
   - **Equipment Required**: None
   - **Total Work Units**: 350 units
   - **Additional Labor Required**: 6 workers
   - **Note**: Additional workers are required to start the task but do not contribute to productivity.
   - **Each productive worker can complete 1 unit per hour.**

---

### **Site Constraints**

- **Equipment Limitations**: Due to space limitations on the site, you can have a maximum of **3 cranes** and **3 concrete mixers** on site at any time.
- **Note**: If you select more than the allowed number of equipment, you will receive a warning.

### **Objectives**

- Complete the project within the given timeframe of **2 months (April-May)**.
- Stay within the allocated budget of **5 million**.
- Efficiently manage resources (equipment and labor).
- **Equipment purchase price is the cost of renting for 30 days**.
- **Equipment productivity is zero if inadequate labor is hired**.
- **An early finish bonus of $5,000 per day is awarded for each day saved**.
- **Your performance will be evaluated based on cost and project duration**.

### **Scoring System**

Your final score is out of **100 points** and is calculated under three schemes:

1. **Cost-Focused Scheme**:
   - **Weight**: 80% Cost, 20% Time
   - **Formula**:
     Total Score = (Budget Score × 0.8) + (Time Score × 0.2)

2. **Balanced Scheme**:
   - **Weight**: 50% Cost, 50% Time
   - **Formula**:
     Total Score = (Budget Score × 0.5) + (Time Score × 0.5)

3. **Time-Focused Scheme**:
   - **Weight**: 20% Cost, 80% Time
   - **Formula**:
     Total Score = (Budget Score × 0.2) + (Time Score × 0.8)

- **Budget Score**:
  - Calculated based on the **remaining budget** at the end of the project relative to the **initial budget**.
  - **Formula**:
    Budget Score = (Remaining Budget / Initial Budget) × 50

- **Time Score**:
  - Calculated based on the **number of days saved** compared to the total project days.
  - Includes the **early finish bonus** of **$5,000 per day**.
  - **Formula**:
    Time Score = ((Total Days - Days Used) / Total Days) × 50

---

### **Rules**

- You can **purchase or rent multiple units** of the same equipment type at the project start.
- **Rented equipment can be rented or returned at any time** during the project.
- **Purchased equipment cannot be returned**, but is cheaper if used for 30 days or more.
- **Renting equipment incurs an initial fee of $10,000 per unit each time it is rented**.
- You can **modify the workforce** during the project (increase or decrease the number of workers).
- **Workers are paid daily**, and you can hire or remove them during the project.
- **Additional workers are required for tasks but do not contribute to productivity**.
- Some tasks have **maintenance workers** who do not contribute to productivity.
- **Equipment requires operators; if you don't have enough workers to operate equipment, its productivity will be zero**.
- **Tasks will not start if less than 1 hour remains in the workday.**
- Making strategic decisions on equipment and labor will affect your project's success.
- **Your final score will be based on your total expenses, project duration, and early finish bonus**.

---

## **Ready to Begin?**

Click the **"Start Game"** button below when you're ready to begin the game.

"""

def main_game():
    # Initialize the project
    project_budget = 5000000
    project_timeline = 60  # Days (April-May)
    project = Project(project_budget, project_timeline)
    
    # Initialize tasks
    initialize_tasks_for_display(project)
    
    # Display introduction and tasks
    display(Markdown(introduction))
    
    # Start Game button
    start_button = widgets.Button(description="Start Game", button_style='success')
    display(start_button)  # Ensure the button is displayed
    
    def on_start_button_clicked(b):
        clear_output()
        # Proceed to equipment selection
        equipment_selection_phase(project)
    
    start_button.on_click(on_start_button_clicked)

# Equipment class
class Equipment:
    def __init__(self, name, eq_type, capacity, productivity_rate, rental_cost_per_day, operating_cost_per_hour, labor_required):
        self.name = name
        self.eq_type = eq_type  # Equipment type (e.g., 'Crane', 'Concrete Mixer')
        self.capacity = capacity
        self.productivity_rate = productivity_rate  # Units per hour
        self.rental_cost_per_day = rental_cost_per_day
        self.operating_cost_per_hour = operating_cost_per_hour
        self.labor_required = labor_required  # Number of workers needed to operate equipment
        self.quantity_purchased = 0
        self.quantity_rented = 0
        self.days_rented = 0  # Track the number of days rented
        # Purchase price is renting cost for 30 days
        self.purchase_cost = self.rental_cost_per_day * 30
        self.rental_fee = 10000  # Initial rental fee per unit

# Task class
class Task:
    def __init__(self, name, quantity, equipment_required, labor_required, sequence, per_worker_productivity=None, maintenance_workers=0):
        self.name = name
        self.quantity = quantity  # Total work units to complete
        self.equipment_required = equipment_required  # Dictionary of equipment types required
        self.labor_required = labor_required  # Additional workers required (do not contribute to productivity)
        self.sequence = sequence  # Order in which tasks must be completed
        self.per_worker_productivity = per_worker_productivity  # Units per hour per worker, if applicable
        self.is_completed = False
        self.progress = 0  # Units completed
        self.progress_history = [0]  # For graphing (start with day 0)
        self.maintenance_workers = maintenance_workers  # Workers needed for maintenance (non-productive)

# Project class
class Project:
    def __init__(self, budget, total_days):
        self.initial_budget = budget  # Save initial budget for scoring
        self.budget = budget
        self.total_days = total_days
        self.current_day = 0  # Start at day 0
        self.expenses = 0
        self.tasks = []
        self.equipment_inventory = []
        self.workers = []
        self.daily_report = []
        self.logs = []
        self.budget_history = [budget]  # For graphing (start with initial budget)
        self.available_equipment = []  # For dynamic equipment rental
        
    def add_task(self, task):
        self.tasks.append(task)
        self.tasks.sort(key=lambda x: x.sequence)  # Ensure tasks are in sequence
        
    def can_execute_task(self, task):
        # Check if previous tasks are completed
        if task.sequence > 1:
            previous_task = next((t for t in self.tasks if t.sequence == task.sequence - 1), None)
            if previous_task and not previous_task.is_completed:
                return False  # Can't proceed until previous task is completed
        # Check equipment availability for the task
        for eq_type in task.equipment_required.keys():
            available_qty = self.get_total_equipment_qty_by_type(eq_type)
            if available_qty < 1:
                return False
        # Check labor
        total_labor_required = task.labor_required + task.maintenance_workers
        if len(self.workers) < total_labor_required:
            return False
        return True

    def get_total_equipment_qty_by_type(self, equipment_type):
        total_qty = sum((eq.quantity_purchased + eq.quantity_rented) for eq in self.equipment_inventory if eq.eq_type == equipment_type)
        return total_qty

    def get_equipment_labor_required(self, task):
        total_labor = 0
        for eq_type in task.equipment_required.keys():
            eqs = [eq for eq in self.equipment_inventory if eq.eq_type == eq_type]
            for eq in eqs:
                total_qty = eq.quantity_purchased + eq.quantity_rented
                total_labor += total_qty * eq.labor_required
        return total_labor

    def get_task_equipment_productivity(self, task, available_workers):
        if not task.equipment_required:
            return 0  # No equipment productivity
        total_productivity = 0
        for eq_type in task.equipment_required.keys():
            eqs = [eq for eq in self.equipment_inventory if eq.eq_type == eq_type]
            for eq in eqs:
                total_units = eq.quantity_purchased + eq.quantity_rented
                for _ in range(total_units):
                    if available_workers >= eq.labor_required:
                        available_workers -= eq.labor_required
                        total_productivity += eq.productivity_rate
                    else:
                        # Not enough workers to operate this equipment unit
                        break
        return total_productivity

    def calculate_task_equipment_cost(self, task, equipment_units_used):
        total_cost = 0
        for eq_type in task.equipment_required.keys():
            eqs = [eq for eq in self.equipment_inventory if eq.eq_type == eq_type]
            for eq in eqs:
                if eq.name in equipment_units_used:
                    units_used = equipment_units_used[eq.name]
                    total_cost += eq.operating_cost_per_hour * 8 * units_used  # Assuming 8-hour workday
        return total_cost

    def calculate_task_labor_cost(self, labor_used):
        wage_per_worker = 200  # Daily wage
        labor_cost = wage_per_worker * labor_used
        return labor_cost

    def return_rented_equipment(self, equipment_name, quantity):
        eq = next((e for e in self.equipment_inventory if e.name == equipment_name), None)
        if eq and eq.quantity_rented >= quantity:
            eq.quantity_rented -= quantity
            print(f"Returned {quantity} units of {eq.name}.")
            return True
        print(f"Unable to return {quantity} units of {equipment_name}.")
        return False

    def rent_equipment(self, equipment_name, quantity):
        eq = next((e for e in self.available_equipment if e.name == equipment_name), None)
        if eq and (self.get_total_equipment_qty_by_type(eq.eq_type) + quantity) <= 3:
            total_rental_fee = (eq.rental_fee + eq.rental_cost_per_day) * quantity
            if self.budget >= total_rental_fee:
                self.budget -= eq.rental_fee * quantity
                existing_eq = next((e for e in self.equipment_inventory if e.name == equipment_name), None)
                if existing_eq:
                    existing_eq.quantity_rented += quantity
                else:
                    # Create a new equipment instance for the inventory
                    new_eq = Equipment(eq.name, eq.eq_type, eq.capacity, eq.productivity_rate, eq.rental_cost_per_day, eq.operating_cost_per_hour, eq.labor_required)
                    new_eq.quantity_rented = quantity
                    self.equipment_inventory.append(new_eq)
                print(f"Rented {quantity} units of {eq.name}. Initial rental fee: ${eq.rental_fee * quantity}")
                return True
            else:
                print(f"Not enough budget to rent {quantity} units of {eq.name}.")
        else:
            print(f"Unable to rent {quantity} units of {equipment_name}. Check site constraints or equipment availability.")
        return False

    def modify_workforce(self, change_in_workers):
        if change_in_workers > 0:
            # Hiring additional workers
            self.workers.extend([200 for _ in range(change_in_workers)])
            print(f"Hired {change_in_workers} additional workers.")
        elif change_in_workers < 0:
            # Laying off workers
            num_workers_to_remove = min(-change_in_workers, len(self.workers))
            for _ in range(num_workers_to_remove):
                self.workers.pop()
            print(f"Laid off {num_workers_to_remove} workers.")
        else:
            print("No change in workforce.")

def equipment_selection_phase(project):
    # Define available equipment with types
    available_equipment = [
        Equipment("100-ton Crane", eq_type="Crane", capacity=100, productivity_rate=0.5, rental_cost_per_day=10000, operating_cost_per_hour=500, labor_required=2),
        Equipment("200-ton Crane", eq_type="Crane", capacity=200, productivity_rate=1.0, rental_cost_per_day=18000, operating_cost_per_hour=800, labor_required=3),
        Equipment("Concrete Mixer A", eq_type="Concrete Mixer", capacity=10, productivity_rate=5, rental_cost_per_day=5000, operating_cost_per_hour=200, labor_required=1),
        Equipment("Concrete Mixer B", eq_type="Concrete Mixer", capacity=20, productivity_rate=8, rental_cost_per_day=8000, operating_cost_per_hour=350, labor_required=2),
    ]
    project.available_equipment = available_equipment  # Make available throughout the project
    
    equipment_items = []
    for eq in available_equipment:
        # Create labels
        specs = f"Purchase: ${eq.purchase_cost:.2f}, Rent/day: ${eq.rental_cost_per_day}, Productivity: {eq.productivity_rate} units/hr, Labor Req'd: {eq.labor_required}"
        eq_label = widgets.Label(value=eq.name, layout=widgets.Layout(width='150px'))
        specs_label = widgets.Label(value=specs, layout=widgets.Layout(width='600px'))
        # Input for quantity
        qty_widget = widgets.BoundedIntText(value=0, min=0, max=3, step=1, description='Quantity:', layout=widgets.Layout(width='200px'))
        # Radio buttons for purchase or rent
        purchase_option = widgets.RadioButtons(options=['Purchase', 'Rent'], value='Rent', description='Method:', layout=widgets.Layout(width='200px'))
        # Button to calculate productivity
        calc_button = widgets.Button(description='Calculate Productivity', layout=widgets.Layout(width='200px'))
        # Output for correct answer
        calc_output = widgets.Output(layout=widgets.Layout(max_height='150px', overflow='auto'))
        
        def on_calc_button_clicked(b, eq=eq, qty_widget=qty_widget, calc_output=calc_output):
            with calc_output:
                clear_output()
                qty = qty_widget.value
                if qty > 0:
                    expected_productivity = eq.productivity_rate * qty * 8  # 8-hour day
                    # Prompt player to calculate
                    prompt = widgets.Text(description='Enter daily productivity (units/day):', layout=widgets.Layout(width='350px'))
                    submit_button = widgets.Button(description='Submit')
                    feedback_output = widgets.Output()
                    def on_submit(b):
                        with feedback_output:
                            clear_output()
                            try:
                                user_productivity = float(prompt.value)
                            except ValueError:
                                print("Please enter a valid number.")
                                return
                            correct_productivity = expected_productivity
                            if abs(user_productivity - correct_productivity) > 0.01:
                                print(f"Incorrect productivity. Correct answer: {correct_productivity} units/day.")
                            else:
                                print("Correct productivity!")
                    submit_button.on_click(on_submit)
                    display(prompt, submit_button, feedback_output)
                else:
                    print("Please enter a quantity greater than 0.")
        calc_button.on_click(on_calc_button_clicked)
        # Arrange widgets
        equipment_box = widgets.VBox([
            widgets.HBox([eq_label, specs_label]),
            widgets.HBox([qty_widget, purchase_option]),
            calc_button,
            calc_output
        ], layout=widgets.Layout(border='solid 1px gray', padding='5px', margin='5px'))
        equipment_items.append(equipment_box)
    
    # Display all equipment options
    equipment_selection_box = widgets.VBox(equipment_items)
    display(equipment_selection_box)
    
    submit_equipment_button = widgets.Button(description="Submit Equipment Selection")
    display(submit_equipment_button)
    
    equipment_warning_output = widgets.Output()
    display(equipment_warning_output)
    
    def on_submit_equipment(b):
        total_purchase_cost = 0
        total_cranes = 0
        total_mixers = 0
        for equipment_box in equipment_items:
            eq_label = equipment_box.children[0].children[0]
            qty_widget = equipment_box.children[1].children[0]
            purchase_option = equipment_box.children[1].children[1]
            eq_name = eq_label.value
            eq = next((e for e in available_equipment if e.name == eq_name), None)
            quantity = qty_widget.value
            if quantity > 0:
                if eq.eq_type == 'Crane':
                    total_cranes += quantity
                elif eq.eq_type == 'Concrete Mixer':
                    total_mixers += quantity
                method = purchase_option.value
                if method == 'Purchase':
                    cost = eq.purchase_cost * quantity
                    if project.budget >= cost:
                        project.budget -= cost
                        eq.quantity_purchased += quantity
                        if eq not in project.equipment_inventory:
                            project.equipment_inventory.append(eq)
                        print(f"Purchased {quantity} units of {eq.name} for ${cost:.2f}")
                    else:
                        print(f"Not enough budget to purchase {quantity} units of {eq.name}.")
                elif method == 'Rent':
                    # Apply rental fee
                    total_rental_fee = (eq.rental_fee + eq.rental_cost_per_day) * quantity
                    if project.budget >= total_rental_fee:
                        project.budget -= eq.rental_fee * quantity
                        eq.quantity_rented += quantity
                        if eq not in project.equipment_inventory:
                            project.equipment_inventory.append(eq)
                        print(f"Rented {quantity} units of {eq.name}. Initial rental fee: ${eq.rental_fee * quantity}")
                    else:
                        print(f"Not enough budget to rent {quantity} units of {eq.name}.")
        with equipment_warning_output:
            clear_output()
            if total_cranes > 3:
                print(f"Warning: You have selected {total_cranes} cranes, which exceeds the site limit of 3 cranes.")
            if total_mixers > 3:
                print(f"Warning: You have selected {total_mixers} concrete mixers, which exceeds the site limit of 3 mixers.")
        print(f"Remaining budget after equipment selection: ${project.budget:.2f}")
        submit_equipment_button.disabled = True  # Prevent re-submission
        # Proceed to labor hiring
        labor_hiring_phase(project)
    
    submit_equipment_button.on_click(on_submit_equipment)

def labor_hiring_phase(project):
    print("\nLabor Hiring")
    num_workers_widget = widgets.BoundedIntText(value=10, min=0, max=500, step=1, description='Number of Workers:', layout=widgets.Layout(width='250px'))
    display(num_workers_widget)
    
    submit_workers_button = widgets.Button(description="Submit Worker Hiring")
    display(submit_workers_button)
    
    def on_submit_workers(b):
        num_workers = num_workers_widget.value
        # Workers are paid daily, no upfront cost
        project.workers = [200 for _ in range(num_workers)]  # Wage per worker is $200/day
        print(f"Hired {num_workers} workers.")
        submit_workers_button.disabled = True  # Prevent re-submission
        num_workers_widget.disabled = True
        # Proceed to project execution
        project_execution_phase(project)
    
    submit_workers_button.on_click(on_submit_workers)

def project_execution_phase(project):
    output_area = widgets.Output(layout=widgets.Layout(height='300px', overflow='auto'))
    display(output_area)
    
    # Controls for returning equipment and modifying workforce
    return_eq_button = widgets.Button(description="Return Equipment")
    modify_workers_button = widgets.Button(description="Modify Workforce")
    next_day_button = widgets.Button(description="Next Day")
    rent_eq_button = widgets.Button(description="Rent Equipment")
    
    # Dropdown and inputs
    rented_equipment_names = [eq.name for eq in project.available_equipment]
    rent_eq_dropdown = widgets.Dropdown(options=rented_equipment_names, description='Equipment:', layout=widgets.Layout(width='200px'))
    rent_eq_qty = widgets.BoundedIntText(value=1, min=1, max=3, description='Quantity:', layout=widgets.Layout(width='200px'))
    return_eq_dropdown = widgets.Dropdown(description='Equipment:', layout=widgets.Layout(width='200px'))
    return_eq_qty = widgets.BoundedIntText(value=1, min=1, max=10, description='Quantity:', layout=widgets.Layout(width='200px'))
    modify_workers_widget = widgets.IntText(value=0, description='Change in Workers:', layout=widgets.Layout(width='200px'))
    
    # Update rented equipment options
    def update_return_eq_options():
        rented_equipment = [eq.name for eq in project.equipment_inventory if eq.quantity_rented > 0]
        return_eq_dropdown.options = rented_equipment
        if not rented_equipment:
            return_eq_button.disabled = True
            return_eq_dropdown.disabled = True
            return_eq_qty.disabled = True
        else:
            return_eq_button.disabled = False
            return_eq_dropdown.disabled = False
            return_eq_qty.disabled = False
    
    update_return_eq_options()
    
    # Dashboard output
    dashboard_output = widgets.Output()
    display(dashboard_output)
    
    def update_dashboard():
        with dashboard_output:
            clear_output()
            print("### Project Dashboard")
            print(f"Day {project.current_day}")
            print(f"Remaining Budget: ${project.budget:.2f}")
            print(f"Total Workers Hired: {len(project.workers)}")
            print("Equipment Onsite:")
            for eq in project.equipment_inventory:
                total_qty = eq.quantity_purchased + eq.quantity_rented
                if total_qty > 0:
                    print(f"- {eq.name}: {total_qty} units")
    
    update_dashboard()
    
    def on_return_eq(b):
        equipment_name = return_eq_dropdown.value
        quantity = return_eq_qty.value
        project.return_rented_equipment(equipment_name, quantity)
        # Update dropdown options
        update_return_eq_options()
        update_dashboard()
    
    def on_rent_eq(b):
        equipment_name = rent_eq_dropdown.value
        quantity = rent_eq_qty.value
        if quantity > 0:
            project.rent_equipment(equipment_name, quantity)
            # Update equipment options
            update_return_eq_options()
            update_dashboard()
    
    def on_modify_workers(b):
        change_in_workers = modify_workers_widget.value
        project.modify_workforce(change_in_workers)
        modify_workers_widget.value = 0  # Reset input
        update_dashboard()
    
    # Initialize graphs
    project_figures = initialize_graphs(project)
    
    def on_next_day(b):
        with output_area:
            clear_output(wait=True)
            project.current_day += 1  # Increment day at the start
            
            print(f"Day {project.current_day}:")
            daily_expense = 0
            daily_expense_equipment = 0
            daily_expense_labor = 0
            hours_remaining = 8  # Start with 8-hour workday
            tasks = [task for task in project.tasks if not task.is_completed]
            task_index = 0
            while hours_remaining >= 1 and task_index < len(tasks):
                task = tasks[task_index]
                if project.can_execute_task(task):
                    total_workers = len(project.workers)
                    required_workers = task.labor_required + task.maintenance_workers
                    available_workers = total_workers - required_workers
                    if not task.equipment_required:
                        # Tasks without equipment
                        productive_workers = available_workers
                        if productive_workers <= 0:
                            print(f"- {task.name}: No productive workers available. Progress cannot be made today.")
                            labor_used = total_workers
                            time_spent = 0
                        else:
                            if task.per_worker_productivity is None:
                                task.per_worker_productivity = 1  # Default productivity
                            productivity_per_hour = productive_workers * task.per_worker_productivity
                            time_to_complete = (task.quantity - task.progress) / productivity_per_hour
                            time_spent = min(hours_remaining, time_to_complete)
                            if time_spent < 1 and task.progress == 0:
                                # If less than 1 hour remains, do not proceed to next task
                                break
                            progress = productivity_per_hour * time_spent
                            task.progress += progress
                            hours_remaining -= time_spent
                            labor_used = total_workers
                            if task.progress >= task.quantity:
                                task.is_completed = True
                                task.progress = task.quantity
                                print(f"- {task.name}: Completed. Total progress: {task.progress}/{task.quantity}")
                            else:
                                print(f"- {task.name}: Progressed {progress:.2f} units. Total progress: {task.progress}/{task.quantity}")
                        labor_cost = project.calculate_task_labor_cost(labor_used)
                        daily_expense_labor += labor_cost * (max(time_spent, hours_remaining) / 8)
                    else:
                        # Tasks with equipment
                        equipment_units_used = {}
                        equipment_productivity = 0
                        total_equipment_labor_required = 0
                        for eq_type in task.equipment_required.keys():
                            eqs = [eq for eq in project.equipment_inventory if eq.eq_type == eq_type]
                            for eq in eqs:
                                total_units = eq.quantity_purchased + eq.quantity_rented
                                units_used = 0
                                for _ in range(total_units):
                                    if available_workers >= eq.labor_required:
                                        available_workers -= eq.labor_required
                                        equipment_productivity += eq.productivity_rate
                                        total_equipment_labor_required += eq.labor_required
                                        units_used += 1
                                    else:
                                        break
                                equipment_units_used[eq.name] = units_used
                        if equipment_productivity == 0:
                            print(f"- {task.name}: No equipment units can be operated due to insufficient labor.")
                            labor_used = total_workers
                            time_spent = 0
                        else:
                            productivity_per_hour = equipment_productivity
                            time_to_complete = (task.quantity - task.progress) / productivity_per_hour
                            time_spent = min(hours_remaining, time_to_complete)
                            if time_spent < 1 and task.progress == 0:
                                # If less than 1 hour remains, do not proceed to next task
                                break
                            progress = productivity_per_hour * time_spent
                            task.progress += progress
                            hours_remaining -= time_spent
                            labor_used = total_workers
                            if task.progress >= task.quantity:
                                task.is_completed = True
                                task.progress = task.quantity
                                print(f"- {task.name}: Completed. Total progress: {task.progress}/{task.quantity}")
                            else:
                                print(f"- {task.name}: Progressed {progress:.2f} units. Total progress: {task.progress}/{task.quantity}")
                            equipment_cost = project.calculate_task_equipment_cost(task, equipment_units_used)
                            daily_expense_equipment += equipment_cost * (time_spent / 8)
                        labor_cost = project.calculate_task_labor_cost(labor_used)
                        daily_expense_labor += labor_cost * (max(time_spent, hours_remaining) / 8)
                else:
                    print(f"- {task.name}: Cannot proceed due to insufficient resources or previous tasks incomplete.")
                task_index += 1
            # Update progress history for all tasks
            for task in project.tasks:
                task.progress_history.append(task.progress)
            # Rental costs for rented equipment
            rental_costs = sum(eq.rental_cost_per_day * eq.quantity_rented for eq in project.equipment_inventory)
            daily_expense_equipment += rental_costs
            for eq in project.equipment_inventory:
                if eq.quantity_rented > 0:
                    eq.days_rented += 1
            # Labor costs for idle workers
            total_labor_cost = project.calculate_task_labor_cost(len(project.workers))
            daily_expense_labor += total_labor_cost * (hours_remaining / 8)  # Cost for idle time
            daily_expense = daily_expense_equipment + daily_expense_labor
            project.budget -= daily_expense
            project.expenses += daily_expense
            print(f"Daily expenses: Equipment ${daily_expense_equipment:.2f} + Labor ${daily_expense_labor:.2f} = Total ${daily_expense:.2f}")
            print(f"Remaining budget: ${project.budget:.2f}")
            if project.budget < 0:
                print("Budget exceeded. Project failed.")
                next_day_button.disabled = True
                return_eq_button.disabled = True
                modify_workers_button.disabled = True
                rent_eq_button.disabled = True
                display_final_evaluation(project)
                return
            project.budget_history.append(project.budget)
            # Update equipment options in case equipment was returned
            update_return_eq_options()
            # Update dashboard
            update_dashboard()
            # Update graphs
            update_graphs(project_figures, project)
            # Check for project completion
            if all(task.is_completed for task in project.tasks):
                print(f"\nProject completed successfully on day {project.current_day}!")
                next_day_button.disabled = True
                return_eq_button.disabled = True
                modify_workers_button.disabled = True
                rent_eq_button.disabled = True
                display_final_evaluation(project)
                return
    
    return_eq_button.on_click(on_return_eq)
    modify_workers_button.on_click(on_modify_workers)
    next_day_button.on_click(on_next_day)
    rent_eq_button.on_click(on_rent_eq)
    
    # Separate controls into sections
    equipment_controls = widgets.VBox([
        widgets.Label(value="Equipment Management", layout=widgets.Layout(font_weight='bold')),
        widgets.HBox([rent_eq_dropdown, rent_eq_qty, rent_eq_button]),
        widgets.HBox([return_eq_dropdown, return_eq_qty, return_eq_button])
    ], layout=widgets.Layout(border='solid 1px gray', padding='5px', margin='5px', width='600px'))
    workforce_controls = widgets.VBox([
        widgets.Label(value="Workforce Management", layout=widgets.Layout(font_weight='bold')),
        modify_workers_widget,
        modify_workers_button
    ], layout=widgets.Layout(border='solid 1px gray', padding='5px', margin='5px', width='300px'))
    
    controls = widgets.HBox([
        equipment_controls,
        workforce_controls,
        next_day_button
    ])
    display(controls)

def initialize_graphs(project):
    # Create output widgets for the graphs
    progress_output = widgets.Output()
    budget_output = widgets.Output()
    display(widgets.HBox([progress_output, budget_output]))
    project_figures = {'progress_output': progress_output, 'budget_output': budget_output}
    return project_figures

def update_graphs(project_figures, project):
    with project_figures['progress_output']:
        clear_output(wait=True)
        plt.figure(figsize=(6, 4))
        days = range(project.current_day + 1)  # Days from 0 to current_day
        for task in project.tasks:
            # Extend progress history to match the current day
            while len(task.progress_history) < project.current_day + 1:
                task.progress_history.append(task.progress_history[-1])
            progress_percent = [(p / task.quantity) * 100 for p in task.progress_history]
            plt.plot(days, progress_percent, label=task.name)
        plt.xlabel('Day')
        plt.ylabel('Completion Percentage (%)')
        plt.title('Task Completion Over Time')
        plt.legend()
        plt.tight_layout()
        plt.show()
    with project_figures['budget_output']:
        clear_output(wait=True)
        plt.figure(figsize=(6, 4))
        days = range(len(project.budget_history))
        plt.plot(days, project.budget_history)
        plt.xlabel('Day')
        plt.ylabel('Remaining Budget ($)')
        plt.title('Remaining Budget Over Time')
        plt.tight_layout()
        plt.show()

def display_final_evaluation(project):
    if all(task.is_completed for task in project.tasks):
        print(f"\nProject completed in {project.current_day} days with ${project.budget:.2f} remaining.")
        days_used = project.current_day
        budget_used = project.initial_budget - project.budget
        days_saved = project.total_days - days_used
        early_finish_bonus = days_saved * 5000
        project.budget += early_finish_bonus
        print(f"Total Days Used: {days_used}")
        print(f"Total Money Used: ${budget_used:.2f}")
        print(f"Early Finish Bonus: ${early_finish_bonus:.2f}")
        print(f"Final Budget after Bonus: ${project.budget:.2f}")
        # Calculate scores
        budget_score = (project.budget / project.initial_budget) * 50
        time_score = (days_saved / project.total_days) * 50
        # Scoring schemes
        print("\n### Scoring Summary:")
        # Cost-Focused Scheme
        cost_focused_score = budget_score * 0.8 + time_score * 0.2
        print(f"**Cost-Focused Scheme (80% Cost, 20% Time):** {cost_focused_score:.2f}/100")
        # Balanced Scheme
        balanced_score = budget_score * 0.5 + time_score * 0.5
        print(f"**Balanced Scheme (50% Cost, 50% Time):** {balanced_score:.2f}/100")
        # Time-Focused Scheme
        time_focused_score = budget_score * 0.2 + time_score * 0.8
        print(f"**Time-Focused Scheme (20% Cost, 80% Time):** {time_focused_score:.2f}/100")
        print("\n**Detailed Scores:**")
        print(f" - Budget Score: {budget_score:.2f}/50")
        print(f" - Time Score: {time_score:.2f}/50")
    else:
        print(f"\nProject failed. Not all tasks were completed within the timeframe.")
        print("Your Score: 0/100")

# Initialize tasks for display
def initialize_tasks_for_display(project):
    task1 = Task("Site Preparation", quantity=100, equipment_required={}, labor_required=5, per_worker_productivity=1, sequence=1)
    task2 = Task("Placing Beams", quantity=300, equipment_required={"Crane": None}, labor_required=5, sequence=2)
    task3 = Task("Placing Concrete Deck", quantity=2300, equipment_required={"Concrete Mixer": None}, labor_required=8, sequence=3)
    task4 = Task("Applying New Traffic Management System", quantity=180, equipment_required={}, labor_required=0, per_worker_productivity=1, maintenance_workers=5, sequence=4)
    task5 = Task("Installing Railing and Lighting", quantity=350, equipment_required={}, labor_required=6, per_worker_productivity=1, sequence=5)
    
    project.add_task(task1)
    project.add_task(task2)
    project.add_task(task3)
    project.add_task(task4)
    project.add_task(task5)

# Start the game
main_game()


Button(description='Submit Equipment Selection', style=ButtonStyle())

Output()

Remaining budget after equipment selection: $5000000.00

Labor Hiring


BoundedIntText(value=10, description='Number of Workers:', layout=Layout(width='250px'), max=500)

Button(description='Submit Worker Hiring', style=ButtonStyle())

Hired 19 workers.


Output(layout=Layout(height='300px', overflow='auto'))

Output()

Laid off 5 workers.
Rented 3 units of 200-ton Crane. Initial rental fee: $30000
Rented 3 units of Concrete Mixer B. Initial rental fee: $30000
Returned 3 units of 200-ton Crane.
